https://www.trulens.org/trulens_eval/getting_started/quickstarts/groundtruth_evals/

结论：

- 即使使用openai api，运行结果中并没有 `ground truth`等指标
- 使用本地模型，情况也是一样的，结果指标都没有得到 `ground truth`等指标

In [1]:
from trulens_eval import Tru

tru = Tru()
tru.reset_database()

🦑 Tru initialized with db url sqlite:///default.sqlite .
🛑 Secret keys may be written to the database. See the `database_redact_keys` option of `Tru` to prevent this.


In [2]:
%%time

from openai import OpenAI
import os

base_url = "http://ape:3000/v1"
api_key = "sk-bJP6QSnUfjAYeYeE505d3eBf63A643BeB0B8E350Df9b7750"

# base_url= "https://api.bianxie.ai/v1"
# api_key = "sk-FdNf8kpfMG1yy1pU16D8E82592974693Bf40E2Df39117991"

os.environ["OPENAI_API_KEY"] = api_key
os.environ["OPENAI_API_BASE"] = base_url

client = OpenAI(api_key=api_key, base_url=base_url)

stream = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[{"role": "user", "content": "解释下微分中值定理的基本思路"}],
    stream=True,
    stream_options= {"include_usage": True}
)

for chunk in stream:
    if chunk.usage:
        print()
        print(chunk.usage)
    else:
        print(chunk.choices[0].delta.content or "", end="")

微分中值定理是微分学中的一个基本定理，它描述了在一个连续函数上存在某个点，该点的导数等于函数在两个固定点处导数之差与自变量在这两个点之差的比值。

具体来说，微分中值定理可以分为两个部分：
1.拉格朗日中值定理：设函数f(x)在闭区间[a,b]上连续，在开区间(a,b)上可导，则在(a,b)内至少存在一点ξ，使得f'(ξ)等于函数在端点a和b处导数的平均值，即f'(ξ)=(f(b)-f(a))/(b-a)。
2.柯西中值定理：设函数f(x)和g(x)在闭区间[a,b]上连续且在开区间(a,b)内可导，且g(x)在区间内没有零点，则在(a,b)内至少存在一点ξ，使得[f(b)-f(a)]/g(b)-g(a)等于[f'(ξ)]/g'(ξ)。

基本思路是通过函数在两个固定点处导数之差与自变量在这两个点之差的比值来推导出函数在某个点的导数等于这个比值。这个定理在证明不同的情况下有不同的证明方法，但都可以通过中值定理来证明。CPU times: user 120 ms, sys: 11.2 ms, total: 131 ms
Wall time: 5.52 s


In [8]:
%%time

from trulens_eval.tru_custom_app import instrument

class APP:
    @instrument
    def completion(self, prompt):
        completion = client.chat.completions.create(
                # model="gpt-3.5-turbo",
                # model="qwen2",
                model="gpt-4-turbo",
                temperature=0,
                messages=
                [
                    {"role": "user",
                    "content": 
                    f"Please answer the question: {prompt}"
                    }
                ]
                ).choices[0].message.content
        return completion
    
llm_app = APP()

CPU times: user 33 µs, sys: 2 µs, total: 35 µs
Wall time: 38.6 µs


In [9]:
%%time

from trulens_eval import Feedback
from trulens_eval.feedback import GroundTruthAgreement

golden_set = [
    {"query": "who invented the lightbulb?", "response": "Thomas Edison"},
    {"query": "¿quien invento la bombilla?", "response": "Thomas Edison"}
]

f_groundtruth = Feedback(GroundTruthAgreement(golden_set).agreement_measure, name = "Ground Truth").on_input_output()


✅ In Ground Truth, input prompt will be set to __record__.main_input or `Select.RecordInput` .
✅ In Ground Truth, input response will be set to __record__.main_output or `Select.RecordOutput` .
CPU times: user 50.4 ms, sys: 5.49 ms, total: 55.9 ms
Wall time: 55 ms


In [10]:
%%time

# add trulens as a context manager for llm_app
from trulens_eval import TruCustomApp
tru_app = TruCustomApp(llm_app, app_id = 'LLM App v1', feedbacks = [f_groundtruth])

CPU times: user 56 ms, sys: 0 ns, total: 56 ms
Wall time: 58.3 ms


In [11]:
%%time

# Instrumented query engine can operate as a context manager:
with tru_app as recording:
    llm_app.completion("¿quien invento la bombilla?")
    llm_app.completion("who invented the lightbulb?")

CPU times: user 661 ms, sys: 0 ns, total: 661 ms
Wall time: 8.7 s


In [12]:
tru.get_leaderboard(app_ids=[tru_app.app_id])

,latency,total_cost
app_id,,
LLM App v1,4.75,0.000648
